# Pose Detection with OpenPose and Video Feedback

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on input videos and provide feedback for choreography given a test video and reference video.

Adapted from the OpenPose notebook from [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).

The code in this notebook assumes that videos and audio have been preprocessed (i.e. videos are aligned by audio track and trimmed to start at the same time). Refer to the choreo-master repository for preprocessing. Postprocessing is also dependent on trimmed videos and audio.

If you do not mount your Google Drive, make sure to download the output files and/or videos as desired as they will not be stored when the notebook is closed. If you do mount your Google Drive, don't forget to move any important files into your Drive as desired before closing the notebook.

## Install OpenPose and Libraries

In [1]:
import os
from os.path import exists, join, basename, splitext
from IPython.display import YouTubeVideo

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

Selecting previously unselected package libgflags2.2.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [2]:
! pip install moviepy

import numpy as np
import json
import more_itertools as mit

## Perform Pose Detection using OpenPose demo

In [3]:
# Set project name (global), used for naming files. CANNOT be empty.
PROJECT_NAME = "bp"

### Upload your own videos

You can upload them to the session from your local computer, or you can mount your Google Drive and get the videos from Drive. The code below runs command line instructions assuming the file names are in the base directory.

Run OpenPose on the test video

In [10]:
# Make sure to edit PROJECT_NAME above!

# (!) Make sure to rename your test video to be in the format: [PROJECT_NAME]_test.mp4
TEST_VIDEO_NAME = "../" + PROJECT_NAME + "_test.mp4" 
OUTPUT_TEST_FOLDER_NAME = "../output_json_" + PROJECT_NAME + "_test/"
OUTPUT_TEST_VIDEO_NAME = PROJECT_NAME + "_test_output.mp4"

!rm openpose.avi

# Run OpenPose on video; write out keypoints data to JSON and output video to openpose.avi.
!cd openpose && ./build/examples/openpose/openpose.bin --video $TEST_VIDEO_NAME --write_json $OUTPUT_TEST_FOLDER_NAME --display 0  --write_video ../openpose.avi

# Convert the result into MP4.
!ffmpeg -y -loglevel info -i openpose.avi $OUTPUT_TEST_VIDEO_NAME

rm: cannot remove 'openpose.avi': No such file or directory
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 37.377414 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-li

Run OpenPose on the reference video

In [11]:
# (!) Make sure to rename your reference video to be in the format: [PROJECT_NAME]_reference.mp4
REFERENCE_VIDEO_NAME = "../" + PROJECT_NAME + "_reference.mp4" 
OUTPUT_REFERENCE_FOLDER_NAME = "../output_json_" + PROJECT_NAME + "_reference/"
OUTPUT_REFERENCE_VIDEO_NAME = PROJECT_NAME + "_reference_output.mp4"

!rm openpose.avi

# Run OpenPose on video; write out keypoints data to JSON and output video to openpose.avi.
!cd openpose && ./build/examples/openpose/openpose.bin --video $REFERENCE_VIDEO_NAME --write_json $OUTPUT_REFERENCE_FOLDER_NAME --display 0  --write_video ../openpose.avi

# Convert the result into MP4.
!ffmpeg -y -loglevel info -i openpose.avi $OUTPUT_REFERENCE_VIDEO_NAME

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
Empty frame detected, frame number 611 of 686. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 611 of 686. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
Empty frame detected, frame number 611 of 686. In /content/openpose/src/openpose/producer/producer.cpp:checkFrameIntegrity():290
OpenPose demo successfully finished. Total time: 35.600729 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa

### Alternatively, use YouTube videos

Run OpenPose on the test video

In [ ]:
# You can find the video ID in the URL after the v=
# i.e. https://www.youtube.com/watch?v=WrJt8iZkUFY
YOUTUBE_ID = 'WrJt8iZkUFY' 
YouTubeVideo(YOUTUBE_ID)

# No need to change anything here
TEST_VIDEO_NAME = PROJECT_NAME + "_test.mp4"
TEST_VIDEO_PATH = "../" + TEST_VIDEO_NAME
OUTPUT_TEST_FOLDER_NAME = "../output_json_" + PROJECT_NAME + "_test/"
OUTPUT_TEST_VIDEO_NAME = PROJECT_NAME + "_test_output.mp4"

!rm -rf youtube.mp4
# Downloads the YouTube video with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Trim as necessary
START_TIME = "00:00:12.5" # in hours:minutes:seconds
DURATION = 15 # duration of clip
!ffmpeg -y -loglevel info -i youtube.mp4 -ss $START_TIME -t $DURATION $TEST_VIDEO_NAME

!rm openpose.avi

# Run OpenPose on video; write out keypoints data to JSON and output video to openpose.avi.
!cd openpose && ./build/examples/openpose/openpose.bin --video $TEST_VIDEO_PATH --write_json $OUTPUT_TEST_FOLDER_NAME --display 0  --write_video ../openpose.avi

# Convert the result into MP4.
!ffmpeg -y -loglevel info -i openpose.avi $OUTPUT_TEST_VIDEO_NAME

[youtube] WrJt8iZkUFY: Downloading webpage
[youtube] WrJt8iZkUFY: Downloading js player 408be03a
[youtube] WrJt8iZkUFY: Downloading js player 408be03a
[download] Destination: youtube.mp4
[download] 100% of 5.01MiB in 00:00
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --

Run OpenPose on the reference video

In [ ]:
# You can find the video ID in the URL after the v=
# i.e. https://www.youtube.com/watch?v=WrJt8iZkUFY
YOUTUBE_ID = 'HsmZNJAqWYk' 
YouTubeVideo(YOUTUBE_ID)

# No need to change anything here
REFERENCE_VIDEO_NAME = PROJECT_NAME + "_reference.mp4"
REFERENCE_VIDEO_PATH = "../" + REFERENCE_VIDEO_NAME
OUTPUT_REFERENCE_FOLDER_NAME = "../output_json_" + PROJECT_NAME + "_reference/"
OUTPUT_REFERENCE_VIDEO_NAME = PROJECT_NAME + "_reference_output.mp4"

!rm -rf youtube.mp4
# Downloads the YouTube video with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

# Trim as necessary
START_TIME = "00:01:03" # in hours:minutes:seconds
DURATION = 15 # duration of clip
!ffmpeg -y -loglevel info -i youtube.mp4 -ss $START_TIME -t $DURATION $REFERENCE_VIDEO_NAME

!rm openpose.avi

# Run OpenPose on video; write out keypoints data to JSON and output video to openpose.avi.
!cd openpose && ./build/examples/openpose/openpose.bin --video $REFERENCE_VIDEO_PATH --write_json $OUTPUT_REFERENCE_FOLDER_NAME --display 0  --write_video ../openpose.avi

# Convert the result into MP4.
!ffmpeg -y -loglevel info -i openpose.avi $OUTPUT_REFERENCE_VIDEO_NAME

[youtube] HsmZNJAqWYk: Downloading webpage
[download] Destination: youtube.mp4
[download] 100% of 73.46MiB in 00:04
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis 

## Extract keypoints and process frame-by-frame

### Helper functions from choreo-master repository

In [12]:
# body25_labels.py
# Contains labels for BODY25 keypoints, bones, and angles

HEAD = 0
CHEST = 1
L_SHOULDER = 2
L_ELBOW = 3
L_WRIST = 4
R_SHOULDER = 5
R_ELBOW = 6
R_WRIST = 7
PELVIS = 8
R_HIP = 9
R_KNEE = 10
R_ANKLE = 11
L_HIP = 12
L_KNEE = 13
L_ANKLE = 14
R_FOOT = 19
L_FOOT = 22

BODY25_BONES = [
    [R_KNEE, R_ANKLE],
    [R_HIP, R_KNEE],
    [R_HIP, PELVIS],
    [L_HIP, PELVIS],
    [L_HIP, L_KNEE],
    [L_KNEE, L_ANKLE],
    [PELVIS, CHEST],
    [CHEST, HEAD],
    [R_WRIST, R_ELBOW],
    [R_ELBOW, R_SHOULDER],
    [CHEST, R_SHOULDER],
    [CHEST, L_SHOULDER],
    [L_SHOULDER, L_ELBOW],
    [L_ELBOW, L_WRIST],
    [R_ANKLE, R_FOOT],
    [L_ANKLE, L_FOOT]
]

RIGHT_LOWER_LEG = 0
RIGHT_UPPER_LEG = 1
RIGHT_LEG_JOINT = 2
LEFT_LEG_JOINT = 3
LEFT_UPPER_LEG = 4
LEFT_LOWER_LEG = 5
SPINE = 6
HEAD_TO_CHEST = 7
RIGHT_LOWER_ARM = 8
RIGHT_UPPER_ARM = 9
RIGHT_SHOULDER_JOINT = 10
LEFT_SHOULDER_JOINT = 11
LEFT_UPPER_ARM = 12
LEFT_LOWER_ARM = 13
RIGHT_FOOT = 14
LEFT_FOOT = 15

BODY25_ANGLES = [
    [RIGHT_LOWER_LEG, RIGHT_UPPER_LEG],
    [LEFT_LOWER_LEG, LEFT_UPPER_LEG],
    [LEFT_LEG_JOINT, LEFT_UPPER_LEG], # less important
    [RIGHT_LEG_JOINT, RIGHT_UPPER_LEG], # less important
    [SPINE, HEAD_TO_CHEST],
    [RIGHT_LOWER_ARM, RIGHT_UPPER_ARM],
    [RIGHT_SHOULDER_JOINT, RIGHT_UPPER_ARM],
    [RIGHT_SHOULDER_JOINT, HEAD_TO_CHEST], # less important
    [LEFT_LOWER_ARM, LEFT_UPPER_ARM],
    [LEFT_SHOULDER_JOINT, LEFT_UPPER_ARM],
    [LEFT_SHOULDER_JOINT, HEAD_TO_CHEST], # less important
    [RIGHT_FOOT, RIGHT_LOWER_LEG],
    [LEFT_FOOT, LEFT_LOWER_LEG]
]

BODY25_ANGLES_NAMES = [
    "[RIGHT_LOWER_LEG, RIGHT_UPPER_LEG]",
    "[LEFT_LOWER_LEG, LEFT_UPPER_LEG]",
    "[LEFT_LEG_JOINT, LEFT_UPPER_LEG]", # less important
    "[RIGHT_LEG_JOINT, RIGHT_UPPER_LEG]", # less important
    "[SPINE, HEAD_TO_CHEST]", # possibly take out later
    "[RIGHT_LOWER_ARM, RIGHT_UPPER_ARM]",
    "[RIGHT_SHOULDER_JOINT, RIGHT_UPPER_ARM]",
    "[RIGHT_SHOULDER_JOINT, HEAD_TO_NECK]", # less important
    "[LEFT_LOWER_ARM, LEFT_UPPER_ARM]",
    "[LEFT_SHOULDER_JOINT, LEFT_UPPER_ARM]",
    "[LEFT_SHOULDER_JOINT, HEAD_TO_NECK]", # less important
    "[RIGHT_FOOT, RIGHT_LOWER_LEG]",
    "[LEFT_FOOT, LEFT_LOWER_LEG]"
]

BODY25_ANGLE_ROMS = [
    150, # [RIGHT_LOWER_LEG, RIGHT_UPPER_LEG],
    150, # [LEFT_LOWER_LEG, LEFT_UPPER_LEG],
    100, # [LEFT_LEG_JOINT, LEFT_UPPER_LEG], 
    100, # [RIGHT_LEG_JOINT, RIGHT_UPPER_LEG], 
    90, # [SPINE, HEAD_TO_CHEST],
    170, # [RIGHT_LOWER_ARM, RIGHT_UPPER_ARM],
    170, # [RIGHT_SHOULDER_JOINT, RIGHT_UPPER_ARM],
    50, # [RIGHT_SHOULDER_JOINT, HEAD_TO_CHEST], 
    170, # [LEFT_LOWER_ARM, LEFT_UPPER_ARM],
    170, # [LEFT_SHOULDER_JOINT, LEFT_UPPER_ARM],
    50, # [LEFT_SHOULDER_JOINT, HEAD_TO_CHEST], 
    90, # [RIGHT_FOOT, RIGHT_LOWER_LEG],
    90, # [LEFT_FOOT, LEFT_LOWER_LEG]
]

In [13]:
# analyze.py

def calculate_angles(keypoints, angle_labels, bone_labels, angle_name_labels, debug=False):
    angles = []
    for i, bones in enumerate(angle_labels):
        k1 = keypoints[bone_labels[bones[0]][0]]
        k2 = keypoints[bone_labels[bones[0]][1]]
        k3 = keypoints[bone_labels[bones[1]][0]]
        k4 = keypoints[bone_labels[bones[1]][1]]
        intersection = k1 if (np.equal(k1, k3).all() or np.equal(k1, k4).all()) else k2
        if np.equal(intersection, k1).all():
            vec1 = intersection - k2
        else:
            vec1 = intersection - k1
        
        if np.equal(intersection, k3).all():
            vec2 = intersection - k4
        else:
            vec2 = intersection - k3
        a1 = np.arctan2(vec1[1], vec1[0])
        a2 = np.arctan2(vec2[1], vec2[0])
        angle = a2 - a1
        if (angle < 0):
            angle += 2 * np.pi
        angles.append(np.degrees(angle))

    for i in range(len(angles)):
        bones = angle_labels[i]
        vec1 = np.array(keypoints[bone_labels[bones[0]][0]] - keypoints[bone_labels[bones[0]][1]])
        vec2 = np.array(keypoints[bone_labels[bones[1]][0]] - keypoints[bone_labels[bones[1]][1]])
        if debug:
            print("{} : {}, vec1 = {}, vec2 = {}".format(angle_name_labels[i], angles[i], vec1, vec2))
    
    return angles

def calculate_relative_angle_scores(kp1, kp2, angle_labels, bone_labels, angle_name_labels, angle_roms, with_ranges_of_motion, debug=False):
    a1 = calculate_angles(kp1, angle_labels, bone_labels, angle_name_labels)
    a2 = calculate_angles(kp2, angle_labels, bone_labels, angle_name_labels)

    sum_diffs = 0
    for i, a in enumerate(angle_labels):
        diff = abs(a1[i] - a2[i])
        if with_ranges_of_motion:
            diff /= (angle_roms[i] * 2)
        sum_diffs += diff
        if debug:
            print("For angle {}, difference is {}".format(angle_name_labels[i], a1[i] - a2[i]))
    return 1 - ((sum_diffs / len(angle_labels)) / 180)

    
def calculate_absolute_angle_scores(kp1, kp2, bone_labels, bone_names=None, debug=False):
    sum_diffs = 0
    bone_count = 0
    for i, b in enumerate(bone_labels):
        v1 = kp1[b[0]] - kp1[b[1]]
        v2 = kp2[b[0]] - kp2[b[1]]
        # skip the keypoint if it wasn't detected
        if v1[1] == 0 or v2[1] == 0:
            continue
        bone_count += 1
        a1 = np.degrees(np.arctan2(v1[1], v1[0]))
        a2 = np.degrees(np.arctan2(v2[1], v2[0]))
        if (a1 < 0):
            a1 += 360
        if (a2 < 0):
            a2 += 360
        diff = a1 - a2
        sum_diffs += abs(diff)
        if debug:
            print("For bone {}:\ntest angle: {},\nref angle: {},\ndifference is {}".format(bone_names[i], a1, a2, diff))
        
    return 1 - ((sum_diffs / bone_count) / 180)

In [14]:
# input_processing_body25.py

def get_keypoints_from_json(frame_filename):
    with open(frame_filename) as f:
        data = json.load(f)
    if len(data['people']) == 0:
        print("No people found in ", frame_filename)
        return []
    all_data = data['people'][0]['pose_keypoints_2d']
    # remove the confidence scores
    kps_only = [i for j, i in enumerate(all_data) if (j+1)%3]
    # split into arrays for each keypoint
    split = len(all_data) / 3
    keypoints = np.array_split(kps_only, split)
    return keypoints

def process_openpose_frames(name, folder_name1, folder_name2, num_frames, base_path = "./"):
    folder_path1, folder_path2 = os.path.join(base_path, folder_name1), os.path.join(base_path, folder_name2)

    num_frames1 = len(os.listdir(folder_path1))
    num_frames2 = len(os.listdir(folder_path2))
    num_frames = min(num_frames1, num_frames2, num_frames)

    rel_no_roms_accs = np.zeros((num_frames,))
    rel_roms_accs = np.zeros((num_frames,))
    abs_accs = np.zeros((num_frames,))

    prev_avg = 0

    for i in range(num_frames):
        number_str = str(i)
        zero_filled_number = number_str.zfill(12) # must have 12 digits
        f_name1 = "{}_test_{}_keypoints.json".format(name, zero_filled_number)

        number_str = str(i)
        zero_filled_number = number_str.zfill(12) # must have 12 digits
        f_name2 = "{}_reference_{}_keypoints.json".format(name, zero_filled_number)
        
        f1_name = os.path.join(folder_path1, f_name1)
        f2_name = os.path.join(folder_path2, f_name2)
        
        kp1 = get_keypoints_from_json(f1_name)
        kp2 = get_keypoints_from_json(f2_name)

        if len(kp1) != 0 and len(kp2) != 0:
            rel_no_roms_acc = calculate_relative_angle_scores(kp1, kp2, BODY25_ANGLES, BODY25_BONES, BODY25_ANGLES_NAMES, BODY25_ANGLE_ROMS, False)
            rel_roms_acc = calculate_relative_angle_scores(kp1, kp2, BODY25_ANGLES, BODY25_BONES, BODY25_ANGLES_NAMES, BODY25_ANGLE_ROMS, True)
            abs_acc = calculate_absolute_angle_scores(kp1, kp2, BODY25_ANGLES, BODY25_BONES)
            prev_avg = (rel_no_roms_acc + rel_roms_acc + abs_acc) / 3
        else:
            acc = prev_avg
        
        rel_no_roms_accs[i] = rel_no_roms_acc
        rel_roms_accs[i] = rel_roms_acc
        abs_accs[i] = abs_acc

    return rel_no_roms_accs, rel_roms_accs, abs_accs

### Extract keypoints

In [15]:
TEST_FOLDER_NAME = "output_json_" + PROJECT_NAME + "_test"
REFERENCE_FOLDER_NAME = "output_json_" + PROJECT_NAME + "_reference"
num_frames = 192 # 144 frames = 6 seconds. multiply time by 24 fps to get # frames.
rel_no_roms_accs, rel_roms_accs, abs_accs = process_openpose_frames(PROJECT_NAME, TEST_FOLDER_NAME, REFERENCE_FOLDER_NAME, num_frames)

In [51]:
# Call this function if you want to write the accuracies to npy files.
def save_accs(rel_no_roms_accs, rel_roms_accs, abs_accs):
  target_file_name = "./" + PROJECT_NAME+"_rel_no_roms_accs_op.npy"
  np.save(target_file_name, rel_no_roms_accs)
  target_file_name = "./" + PROJECT_NAME+"_rel_roms_accs_op.npy"
  np.save(target_file_name, rel_roms_accs)
  target_file_name = "./" + PROJECT_NAME+"_abs_accs_op.npy"
  np.save(target_file_name, abs_accs)

In [52]:
save_accs(rel_no_roms_accs, rel_roms_accs, abs_accs)

## Score analysis

### Helper functions from choreo-master repository

In [16]:
# score_analysis.py

# Helper function to calculate moving average of size w with stride s
def moving_average(x, w, s):
    all_avgs = np.convolve(x, np.ones(w), 'valid') / w
    return all_avgs[::s]

# Helper function to find ranges of consecutive numbers
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]

# Flag sections with high amounts of differences, i.e. scores above a certain threshold.
# Default threshold is based on 25th percentile of scores.
def find_problem_times(accs, fps, percentile = 25):
    # Set a threshold for percentage of losses that are too high
    threshold = np.percentile(accs, percentile)
    
    # Average scores over 0.5 second increments (e.g. for 24fps, 12 frames/half sec)
    avgs = moving_average(accs, int(fps/2), int(fps/2))
    
    # Combine contiguous increments that are above the threshold = sections that are wrong
    below_threshold = np.where(avgs < threshold, avgs, 0)
    problems = np.nonzero(below_threshold)
    sections = list(find_ranges(problems[0]))
    
    # Convert average index to seconds in video.
    # e.g. problems = [1, 4, 9] -> [0.5, 2, 4.5]
    for i, section in enumerate(sections):
        if type(section) is tuple:
            sections[i] = (section[0]/2, section[1]/2)
        else:
            sections[i] = (section/2, section/2 + 0.5)

    return sections


# Returns a list of the top n places for improvement in the form of starting 0.5 second section timestamp. 
# By default, the function will return the top 5 half second sections that had the lowest scores.
def find_top_n_problems(accs, fps, n=5):
    avgs = moving_average(accs, int(fps/2), int(fps/2))

    # Get top n local minimums in the score function (the worst n half second sections).
    top_n = np.argpartition((-1*avgs), -1*n)[-1*n:]

    # Get the indices of the minimums in descending order.
    top_n = top_n[np.argsort(-1*avgs[top_n])]

    # Convert index to timestamp, i.e. index of min / 2.
    top_n = np.divide(top_n, 2)

    return top_n

# Gets the corresponding frame for the given timestamp.
def get_image_with_timestamp(timestamp):
    # Convert index to frame number, i.e. timestamp * fps.
    frame_number = timestamp*fps

    filename = "frame_{}".format(frame_number)

# Generate feedback report with the problem sections in the 25th percentile and top 10 sections for improvement.
def generate_feedback_report(name, accs, fps):
    report = []
    report.append("Report for {}".format(name))
    report.append("------------------------------\n")

    problem_times = find_problem_times(accs, fps)
    report.append("Here are the sections that were most different from the reference:")
    for time in problem_times:
        report.append("> {} - {} seconds".format(time[0], time[1]))

    report.append("\n----------------------------\n")

    top_5_times = find_top_n_problems(accs, fps)
    report.append("Here are the top 5 most different 0.5 second sections:")
    for time in top_5_times:
        report.append("> {} - {} seconds".format(time, time+0.5))

    report.append("\n------------------------------")

    return ("\n".join(report))

### Get score analysis

In [55]:
# Pass in rel_no_roms_accs, rel_roms_accs, or abs_accs as accs into the following functions.

def get_feedback_report(accs, fps=24, save=False):
    report = generate_feedback_report(PROJECT_NAME, accs, fps)
    print(report)
    if save:
      text_file = open(os.path.join("./", "{}_report.txt".format(PROJECT_NAME)), "w")
      n = text_file.write(report)
      text_file.close()

def get_problem_times(accs, fps, percentile=25):
    return find_problem_times(accs, fps, percentile)
    
def get_top_n_problems(accs, fps, n=5):
    return find_top_n_problems(accs, fps, n)

In [59]:
get_feedback_report(abs_accs)

Report for bp
------------------------------

Here are the sections that were most different from the reference:
> 4.0 - 4.5 seconds
> 6.0 - 6.5 seconds

----------------------------

Here are the top 5 most different 0.5 second sections:
> 3.0 - 3.5 seconds
> 6.5 - 7.0 seconds
> 4.0 - 4.5 seconds
> 6.0 - 6.5 seconds
> 4.5 - 5.0 seconds

------------------------------


## Video post processing

### Helper functions from choreo-master repository

In [19]:
# postprocess.py

from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip, CompositeVideoClip, clips_array
import cv2

def side_by_side_videos(test_video_file_path, ref_video_file_path, target_file_path):
    # aiming for 800 x 1200 pixels
    w0 = 800
    h0 = 1200
    desired_ratio = w0/h0
    
    clip1 = VideoFileClip(test_video_file_path)
    clip2 = VideoFileClip(ref_video_file_path)

    w1 = clip1.w
    h1 = clip1.h
    if w1/h1 > desired_ratio:
        # scale width
        clip1 = clip1.resize(w0/w1)
    else:
        # scale height
        clip1 = clip1.resize(h0/h1)

    w2 = clip2.w
    h2 = clip2.h
    if w2/h2 > desired_ratio:
        clip2 = clip2.resize(w0/w1)
    else:
        clip2 = clip2.resize(h0/h2)
    video = clips_array([[clip1,clip2]])
    video.write_videofile(target_file_path)

def add_audio_to_video(video_file_path, audio_file_path, target_file_path):
    videoclip = VideoFileClip(video_file_path)
    audioclip = AudioFileClip(audio_file_path)
    audioclip = audioclip.subclip(0, videoclip.duration)

    new_audioclip = CompositeAudioClip([audioclip])
    videoclip2 = videoclip.set_audio(new_audioclip)
    videoclip2.write_videofile(target_file_path, codec="libx264", audio_codec="aac")

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3547136/45929032 bytes (7.7%)7577600/45929032 bytes (16.5%)11706368/45929032 bytes (25.5%)15704064/45929032 bytes (34.2%)19988480/45929032 bytes (43.5%)24133632/45929032 bytes (52.5%)28082176/45929032 bytes (61.1%)32006144/45929032 bytes (69.7%)35905536/45929032 bytes (78.2%)40050688/45929032 bytes (87.2%)44171264/45929032 bytes (96.2%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

### Generate output video with audio

In [20]:
side_by_side_videos(
    test_video_file_path="{}_test_output.mp4".format(PROJECT_NAME),
    ref_video_file_path="{}_reference_output.mp4".format(PROJECT_NAME),
    target_file_path="{}_{}".format(PROJECT_NAME, "op_comparison.mp4")
)

[MoviePy] >>>> Building video bp_op_comparison.mp4
[MoviePy] Writing video bp_op_comparison.mp4


100%|██████████| 612/612 [01:32<00:00,  6.61it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: bp_op_comparison.mp4 



In [21]:
add_audio_to_video(
    video_file_path="{}_{}".format(PROJECT_NAME, "op_comparison.mp4"),
    audio_file_path="{}_audio.mp3".format(PROJECT_NAME),
    target_file_path="{}_{}".format(PROJECT_NAME, "op_comparison_with_audio.mp4")
)

[MoviePy] >>>> Building video bp_op_comparison_with_audio.mp4
[MoviePy] Writing audio in bp_op_comparison_with_audioTEMP_MPY_wvf_snd.mp4


100%|██████████| 450/450 [00:00<00:00, 470.14it/s]

[MoviePy] Done.
[MoviePy] Writing video bp_op_comparison_with_audio.mp4



100%|█████████▉| 612/613 [00:47<00:00, 12.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: bp_op_comparison_with_audio.mp4 



In [22]:
# Use this function to see the output video.
# Colab cannot really handle large memory files, so try to keep these videos short.
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
filename = "{}_{}".format(PROJECT_NAME, "op_comparison_with_audio.mp4")
show_local_mp4_video(filename, width=960, height=720)